# TimeAffineレイヤを確認する
TimeAffineレイヤは、Affineレイヤを時間方向に結合していくレイヤである

In [1]:
import numpy as np
from common.layers import Affine

In [2]:
class TimeAffine:
    def __init__(self, W, b):
        
        # パラメータのリスト
        self.params = [W, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
 
        self.x = None

    def forward(self, x):
        """
        順伝播計算
        x : 入力データ
        """
        N, T, D = x.shape # バッチサイズ、時間数、前層のノード数
        W, b = self.params

        # 全ての時刻について、一度でAffineの順伝播計算を行う
        rx = x.reshape(N*T, -1)
        out = np.dot(rx, W) + b # 行列の積 + バイアス
        
        # xを保持
        self.x = x
        
        return out.reshape(N, T, -1)

    def backward(self, dout):
        """
        逆伝播計算
        """
        x = self.x
        N, T, D = x.shape # バッチサイズ、時間数、前層のノード数
        W, b = self.params

        # 全ての時刻について、一度でAffineの逆伝播計算を行う
        dout = dout.reshape(N*T, -1)
        rx = x.reshape(N*T, -1)
        db = np.sum(dout, axis=0) # バイアスの勾配
        dW = np.dot(rx.T, dout) # 重みWの勾配
        dx = np.dot(dout, W.T) # 前層へ伝える勾配
        dx = dx.reshape(*x.shape)

        self.grads[0][:] = dW # 同じメモリ位置に代入
        self.grads[1][:] = db # 同じメモリ位置に代入
        
        return dx

In [3]:
np.random.seed(1234)
D = 1 # 入力層のノード数
H = 5 # 中間層のノード数
W = np.random.randn(D, H)
b = np.zeros(H)

# オブジェクトの生成
time_affine = TimeAffine(W, b)
print("id of time_affine.grads[0]", id(time_affine.grads[0]))
print("id of time_affine.grads[1]", id(time_affine.grads[1]))
print()

# 順伝播計算
N = 4 # バッチサイズ
T = 5 # 時間数
x = np.random.randn(N, T, D)
out = time_affine.forward(x)
# print("out=", out)
# print()

# 逆伝播計算
dout = np.random.randn(N, T, H)
dx = time_affine.backward(dout)
# print("dx=", dx)
# print()

print("id of time_affine.grads[0]", id(time_affine.grads[0]))
print("id of time_affine.grads[1]", id(time_affine.grads[1]))
print()


id of time_affine.grads[0] 4586215824
id of time_affine.grads[1] 4586215984

id of time_affine.grads[0] 4586215824
id of time_affine.grads[1] 4586215984

